In [1]:
import cobra
from openpyxl import load_workbook, styles, Workbook
import re
import pandas as pd

# Load model
model_path = "P://git/gem_sco/modelS.xml"
# xl_fn = "P://git/gem_sco/modelS.xlsx"
model = cobra.io.read_sbml_model(model_path)
# wb = load_workbook(xl_fn)


In [3]:
with model:
    #model.reactions.NH4t.bounds = (0, 1000)
    #model.reactions.NH4t.bounds = (0, 1000)
    reaction_list = []#, "ICL"]
    reaction_list_neg = []
    for r_id in reaction_list:
        model.reactions.get_by_id(r_id).upper_bound = 0.000
    for r_id in reaction_list_neg:
        model.reactions.get_by_id(r_id).lower_bound = -0.00
    
    
    solution0 = model.optimize()
    model.reactions.EX_glc_e.bounds = (0, 1000)
    model.reactions.EX_asp__L_e.bounds = (-1, 1000)
    model.reactions.EX_o2_e.bounds = (-2.2, 1000)
    #model.reactions.NH4t.bounds = (0, 1000)
    #model.reactions.NH4t.bounds = (0, 1000)

    
    solution1 = model.optimize()
    model.reactions.EX_asp__L_e.bounds = (0, 1000)
    model.reactions.EX_asn__L_e.bounds = (-1, 1000)
    solution2 = model.optimize()
    
    model.reactions.EX_asn__L_e.bounds = (0, 1000)
    model.reactions.EX_leu__L_e.bounds = (-1, 1000)
    solution3 = model.optimize()
    
    
    model.reactions.EX_leu__L_e.bounds = (0, 1000)
    model.reactions.EX_rib__D_e.bounds = (-1, 1000)
    solution4 = model.optimize()
    
    
        
    model.reactions.EX_rib__D_e.bounds = (0, 1000)
    model.reactions.EX_gly_e.bounds = (-1, 1000)
    solution5 = model.optimize()
    
    
    print(solution0.f)
    print(solution1.f)
    print(solution2.f)
    print(solution3.f)
    print(solution4.f)
    print(solution5.f)

0.07539807474112581
0.03976982001856321
0.039838717611050546
0.05177378604531179
0.07548266127086356
0.007206002905901076


In [6]:
import pandas as pd
diff = solution1.fluxes.subtract(solution2.fluxes)
print(diff.sort_values())

df = pd.DataFrame()
df["glucose"] = solution0.fluxes
df["asp"] = solution1.fluxes
df["asn"] = solution2.fluxes
df["leu"] = solution3.fluxes
df["rib"] = solution4.fluxes
df["glysine"] = solution5.fluxes
df["diff"] = diff
df["reduced_costs"] = solution1.reduced_costs

df_idx0 = df[abs(df["asp"])>1e-4]
df_idx1 = df_idx0[abs(df_idx0["asn"])<=1e-4]
df_idx2 = df_idx1[abs(df_idx1["leu"])<=1e-4]
df_idx2 = df_idx2[abs(df_idx2["glucose"])<=1e-4]
df_idx2 = df_idx2[abs(df_idx2["rib"])<=1e-4]
df_idx3 = df_idx2[abs(df_idx2["glysine"])<=1e-4]


df_idx3.sort_values(by = "diff").round(decimals = 4)

H2Ot          -1.001092
ASNt2r        -1.000000
EX_asp__L_e   -1.000000
EX_nh4_e      -0.999388
ASNN          -0.992692
EX_o2_e       -0.002795
CO2t          -0.002723
SUCOAS        -0.001787
ALATA_L       -0.001665
R10612        -0.001559
R00726        -0.001208
PGM           -0.001164
PGK           -0.001067
GLYALDDr      -0.000779
EX_h_e        -0.000564
ME1           -0.000153
PGCD          -0.000097
PSP_L         -0.000097
PSERT         -0.000097
HCO3E         -0.000094
NDPK2         -0.000072
PIt2r         -0.000070
Biomass_SCO   -0.000069
GHMT2         -0.000066
ASPK          -0.000065
FBP           -0.000065
MCOATA        -0.000060
ACCOAC        -0.000060
PRPPS         -0.000057
59            -0.000049
                 ...   
GLYCDx         0.000802
DHAK           0.000802
TPI            0.000917
GAPD           0.001067
ENO            0.001164
ADK4i          0.001208
GLUDyi         0.001246
R00473         0.001559
ACONTa         0.001662
ACONTb         0.001662
ICDHyr         0

,glucose,asp,asn,leu,rib,glysine,diff,reduced_costs
EX_asp__L_e,0.0,-1.0,0.0,0.0,0.0,0.0,-1.0,-0.0977
ASPt2,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0000


In [9]:
with model:
    model.reactions.EX_glc_e.bounds = (0, 1000)
    model.reactions.EX_asp__L_e.bounds = (-1, 1000)
    model.reactions.EX_o2_e.bounds = (-2.2, 1000)
    model.optimize()
    print(model.metabolites.asp__L_c.summary())
    
    model.reactions.EX_asp__L_e.bounds = (0, 1000)
    model.reactions.EX_asn__L_e.bounds = (-1, 1000)
    model.optimize()
    print(model.metabolites.asn__L_c.summary())
    

PRODUCING REACTIONS -- L-Aspartate (asp__L_c)
---------------------------------------------
%       FLUX  RXN ID    REACTION
----  ------  --------  --------------------------------------------------
100%  1       ASPt2     asp__L_e + h_e --> asp__L_c + h_c

CONSUMING REACTIONS -- L-Aspartate (asp__L_c)
---------------------------------------------
%       FLUX  RXN ID    REACTION
----  ------  --------  --------------------------------------------------
91%   0.906   ASPTA     akg_c + asp__L_c <=> glu__L_c + oaa_c
4%    0.0375  ASPK      asp__L_c + atp_c <=> 4pasp_c + adp_c
2%    0.0156  ASPCT     asp__L_c + cbp_c --> cbasp_c + h_c + pi_c
1%    0.0124  PRASCSi   5aizc_c + asp__L_c + atp_c --> 25aics_c + adp_c...
None
PRODUCING REACTIONS -- L-Asparagine (asn__L_c)
----------------------------------------------
%       FLUX  RXN ID    REACTION
----  ------  --------  -------------------------------------
100%   1      ASNt2r    asn__L_e + h_e <=> asn__L_c + h_c

CONSUMING REACTIONS -- L